# Image Study

On our first approach we will divide each tiktok in 10 frames and try to predict the virality from thgere with a CNN from scratch. We wil start preparing the data

In [5]:
from image_utils import remove_mp4_extensions, extract_frame
import numpy as np
import pandas as pd
import os
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'distutils'

In [2]:
extract_frames()
remove_mp4_extensions()

Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_0.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_24.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_47.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_71.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_95.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_118.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_142.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_166.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_189.png
Saved: ../../data/inputs/frames\6875317312082201857.mp4_frame_213.png
Saved: ../../data/inputs/frames\6875323773755657474.mp4_frame_0.png
Saved: ../../data/inputs/frames\6875323773755657474.mp4_frame_52.png
Saved: ../../data/inputs/frames\6875323773755657474.mp4_frame_104.png
Saved: ../../data/inputs/frames\6875323773755657474.mp4_frame_157.png
Saved: ../../data/inputs/fram